In [1]:
import os
os.chdir("../..")

In [2]:
from shell.datasets.datasets import get_dataset
import numpy as np
import matplotlib.pyplot as plt
from torchvision.utils import make_grid
from shell.utils.utils import seed_everything, viz_embedding
import torch
import subprocess
import torch.nn as nn
import torch
from omegaconf import DictConfig
from shell.datasets.datasets import get_dataset
from shell.utils.utils import seed_everything
from pprint import pprint
from shell.fleet.network import TopologyGenerator
from shell.models.cnn_soft_lifelong_dynamic import CNNSoftLLDynamic
from shell.models.cnn import CNN
from shell.models.mlp import MLP
from shell.models.mlp_soft_lifelong_dynamic import MLPSoftLLDynamic
from shell.learners.er_dynamic import CompositionalDynamicER
from shell.learners.er_nocomponents import NoComponentsER
from shell.utils.experiment_utils import eval_net
from shell.utils.experiment_utils import setup_experiment
from sklearn.manifold import TSNE
import logging
import seaborn as sns
import pickle
logging.basicConfig(level=logging.INFO)

In [3]:
"""
Two agents trained separately
"""

use_contrastive = False
num_tasks = 10

net_cfg = {
        'depth': 2,
        'layer_size': 64,
        'num_init_tasks': 2,
        'i_size': 28,
        'num_classes': 2,
        'num_tasks': num_tasks,
        'dropout': 0.0,
    }

agent_cfg = {
        'memory_size': 64,
        'use_contrastive': use_contrastive,
    }


alice = CompositionalDynamicER(MLPSoftLLDynamic(**net_cfg), **agent_cfg)
bob = CompositionalDynamicER(MLPSoftLLDynamic(**net_cfg), **agent_cfg)

In [4]:
num_tasks = 2
data_cfg = {
            "dataset_name": "mnist",
            "num_tasks": num_tasks,
            "num_train_per_task": 128,
            "num_val_per_task": 102,
            'remap_labels': True,
            'use_contrastive': use_contrastive,
            "with_replacement": True,
        }
bob_dataset = get_dataset(**data_cfg)
for task_id in range(num_tasks):
    testloaders = {task: torch.utils.data.DataLoader(testset,
                                                                batch_size=128,
                                                                shuffle=False,
                                                                num_workers=0,
                                                                pin_memory=True,
                                                                ) for task, testset in enumerate(bob_dataset.testset[:(task_id+1)])}

    trainloader = torch.utils.data.DataLoader(bob_dataset.trainset[task_id],
                                                            batch_size=64,
                                                            shuffle=True,
                                                            num_workers=0,
                                                            pin_memory=True,
                                                            )
    valloader = torch.utils.data.DataLoader(bob_dataset.valset[task_id],
                                                    batch_size=256,
                                                    shuffle=False,
                                                    num_workers=4,
                                                    pin_memory=True,
                                                    )

    bob.train(trainloader, task_id=task_id, num_epochs=100, testloaders=testloaders,
    valloader=valloader, save_freq=1)

INFO:root:Class sequence: [3 1 6 4]
INFO:root:task 0 :(128, 1, 28, 28)
INFO:root:task 1 :(128, 1, 28, 28)
INFO:root:epochs: 0, training task: 0
INFO:root:	task: 0	loss: 0.697	acc: 0.471
INFO:root:	task: avg	loss: 0.697	acc: 0.471
INFO:root:epochs: 0, training task: 0
INFO:root:	task: 0	loss: 0.697	acc: 0.471
INFO:root:	task: avg	loss: 0.697	acc: 0.471
INFO:root:final components: 2
INFO:root:epochs: 0, training task: 1
INFO:root:	task: 0	loss: 0.697	acc: 0.471
INFO:root:	task: 1	loss: 0.691	acc: 0.506
INFO:root:	task: avg	loss: 0.694	acc: 0.489
INFO:root:epochs: 1, training task: 1
INFO:root:	task: 0	loss: 0.688	acc: 0.471
INFO:root:	task: 1	loss: 0.688	acc: 0.506
INFO:root:	task: avg	loss: 0.688	acc: 0.489
INFO:root:epochs: 2, training task: 1
INFO:root:	task: 0	loss: 0.680	acc: 0.471
INFO:root:	task: 1	loss: 0.685	acc: 0.506
INFO:root:	task: avg	loss: 0.683	acc: 0.489
INFO:root:epochs: 3, training task: 1
INFO:root:	task: 0	loss: 0.670	acc: 0.471
INFO:root:	task: 1	loss: 0.682	acc: 0.

In [5]:
num_tasks = 2
data_cfg = {
            "dataset_name": "mnist",
            "num_tasks": num_tasks,
            "num_train_per_task": 128,
            "num_val_per_task": 102,
            'remap_labels': True,
            'use_contrastive': use_contrastive,
            "with_replacement": True,
        }
alice_dataset = get_dataset(**data_cfg)
for task_id in range(num_tasks):
    testloaders = {task: torch.utils.data.DataLoader(testset,
                                                                batch_size=128,
                                                                shuffle=False,
                                                                num_workers=0,
                                                                pin_memory=True,
                                                                ) for task, testset in enumerate(alice_dataset.testset[:(task_id+1)])}

    trainloader = torch.utils.data.DataLoader(alice_dataset.trainset[task_id],
                                                            batch_size=64,
                                                            shuffle=True,
                                                            num_workers=0,
                                                            pin_memory=True,
                                                            )
    valloader = torch.utils.data.DataLoader(alice_dataset.valset[task_id],
                                                    batch_size=256,
                                                    shuffle=False,
                                                    num_workers=4,
                                                    pin_memory=True,
                                                    )

    alice.train(trainloader, task_id=task_id, num_epochs=100, testloaders=testloaders,
    valloader=valloader, save_freq=1)

INFO:root:Class sequence: [7 8 3 8]
INFO:root:task 0 :(128, 1, 28, 28)
INFO:root:task 1 :(128, 1, 28, 28)
INFO:root:epochs: 0, training task: 0
INFO:root:	task: 0	loss: 0.696	acc: 0.468
INFO:root:	task: avg	loss: 0.696	acc: 0.468
INFO:root:epochs: 0, training task: 0
INFO:root:	task: 0	loss: 0.696	acc: 0.468
INFO:root:	task: avg	loss: 0.696	acc: 0.468
INFO:root:final components: 2
INFO:root:epochs: 0, training task: 1
INFO:root:	task: 0	loss: 0.696	acc: 0.468
INFO:root:	task: 1	loss: 0.697	acc: 0.422
INFO:root:	task: avg	loss: 0.697	acc: 0.445
INFO:root:epochs: 1, training task: 1
INFO:root:	task: 0	loss: 0.689	acc: 0.663
INFO:root:	task: 1	loss: 0.694	acc: 0.491
INFO:root:	task: avg	loss: 0.692	acc: 0.577
INFO:root:epochs: 2, training task: 1
INFO:root:	task: 0	loss: 0.683	acc: 0.748
INFO:root:	task: 1	loss: 0.690	acc: 0.493
INFO:root:	task: avg	loss: 0.686	acc: 0.621
INFO:root:epochs: 3, training task: 1
INFO:root:	task: 0	loss: 0.676	acc: 0.704
INFO:root:	task: 1	loss: 0.686	acc: 0.

In [6]:
bob.net

MLPSoftLLDynamic(
  (structure): ParameterList(
      (0): Parameter containing: [torch.float32 of size 2x2 (GPU 0)]
      (1): Parameter containing: [torch.float32 of size 2x2 (GPU 0)]
      (2): Parameter containing: [torch.float32 of size 2x2 (GPU 0)]
      (3): Parameter containing: [torch.float32 of size 2x2 (GPU 0)]
      (4): Parameter containing: [torch.float32 of size 2x2 (GPU 0)]
      (5): Parameter containing: [torch.float32 of size 2x2 (GPU 0)]
      (6): Parameter containing: [torch.float32 of size 2x2 (GPU 0)]
      (7): Parameter containing: [torch.float32 of size 2x2 (GPU 0)]
      (8): Parameter containing: [torch.float32 of size 2x2 (GPU 0)]
      (9): Parameter containing: [torch.float32 of size 2x2 (GPU 0)]
  )
  (softmax): Softmax(dim=0)
  (components): ModuleList(
    (0-1): 2 x Linear(in_features=64, out_features=64, bias=True)
  )
  (relu): ReLU()
  (dropout): Dropout(p=0.0, inplace=False)
  (random_linear_projection): Linear(in_features=784, out_features=64, b

In [7]:
"""
Manually plug in alice modules to Bob so that bob can solves alice stuff too!
"""

'\nManually plug in alice modules to Bob so that bob can solves alice stuff too!\n'

In [8]:
print(eval_net(bob.net, testloaders))
print(eval_net(alice.net, testloaders))

{0: 0.5539460539460539, 1: 0.610383064516129, 'avg': 0.5821645592310914}
{0: 0.954045954045954, 1: 0.9324596774193549, 'avg': 0.9432528157326545}
